In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql(f"""
CREATE OR REPLACE TABLE {catalog_name}.udp_wcm_gold_vcm_dwh.d_order_method
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)
AS
SELECT
    store_id,
    product_id, 
    CASE WHEN order_method = 'Eton' THEN 'Eton' ELSE 'VCM' END order_method,
    start_date,
    end_date
FROM (
        SELECT
            A.*,
            ROW_NUMBER() OVER (PARTITION BY store_id, product_id, end_date ORDER BY proc_date) RN
            FROM (
                SELECT
                    product_id, store_id, order_method, proc_date, start_date, end_date
                    FROM {catalog_name}.udp_wcm_silver_dim.stg_vm_ordermethod
                ) A
) X WHERE X.RN = 1;
""")